### Import python library

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

### 1. Read data: augment_24group_1620.csv필요

In [2]:
timestep = 3

In [3]:
PCT_lstm_train_X = pd.read_csv("PCT_lstm_final_train_X33.csv")
PCT_lstm_train_y = pd.read_csv("PCT_lstm_final_train_y33.csv")

PCT_lstm_test_X = pd.read_csv("PCT_lstm_final_test_X33.csv")
PCT_lstm_test_y = pd.read_csv("PCT_lstm_final_test_y33.csv")

In [4]:
team = list(PCT_lstm_train_X.T_ID.unique())
year = list(PCT_lstm_train_y.YEAR.unique())

In [5]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

#### 4) input shape로 변경 (row, timestep=2, feature)

ex) 
timestep = 2

* X_train_v 구성예시: [[1 ~ 24경기 데이터, 25 ~ 48경기 데이터], [49 ~ 72경기 데이터, 73 ~ 96경기 데이터] ]  
X_train_v.shape >> (2,2*x)             # x: 각 24group에 대한 변수 개수
* y_train_v 구성예시: 97 ~ 120 경기 승률

=> reshape

* X_train_v.shape >> (2,2,x)  # row, timestep, feature

### 2. 모델 구성(LSTM)
- optimizer: Adam -> lr(learning rate) 조절
- LSTM: 모델이 계속 동일한 결과값이 나올 때, input 뉴런 개수를 늘려야 한다는 글을 읽고 계속 input 노드 개수를 바꿔주면서 모델 생성중
- loss: MSE

- early_stop: patience를 크게하면 과적합 되는 경우가 있어서 최대한 작게 설정해둠
- batch_size: 모델이 계속 동일한 결과값이 나올 때, 데이터가 적어 batch size를 줄여보라는 글을 읽고 1로 설정해둠

In [11]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID','y','y_pred',"shift_PCT_1","shift_PCT_2","shift_PCT_3",'rms','rms0.5'])

idx = 0
for y in year:
    tmp1 = PCT_lstm_train_X[PCT_lstm_train_X["YEAR"] == y]
    tmp2 = PCT_lstm_train_y[PCT_lstm_train_y["YEAR"] == y]
    tmp3 = PCT_lstm_test_X[PCT_lstm_test_X["YEAR"] == y]
    tmp4 = PCT_lstm_test_y[PCT_lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
        y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        X_train_v = X_train.values
        y_train_v = y_train.values

        X_test_v = X_test.values
        y_test_v = y_test.values
        
        X_train_t = X_train_v.reshape(X_train_v.shape[0], timestep,X_train_v.shape[1]//timestep)
        X_test_t = X_test_v.reshape(X_test_v.shape[0], timestep,X_test_v.shape[1]//timestep)
        
        
        ## model
        K.clear_session() 

        model = Sequential()
        optimizer = Adam(lr=0.01)
#         optimizer = RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)

        model.add(LSTM(100,input_shape = (timestep,X_train_v.shape[1]//timestep))) # (timestep, feature)
        model.add(Dense(1)) # output = 1
        model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

        model.summary()
        
#         hist1 = model.fit(X_train_t, y_train_v, epochs=100, batch_size=1, verbose=1)
        
        early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

        hist1 = model.fit(X_train_t, y_train_v, epochs=100,
                  batch_size=2, verbose=1, callbacks=[early_stop])
        ##
        
        model_dict[name] = model
        hist_dict[name] = hist1
        
        y_pred = model.predict(X_test_t)
        rms = sqrt(mean_squared_error(y_test_v, y_pred))
        rms05 = sqrt(mean_squared_error(y_test_v,[0.5]))
        
        
        test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0],
                                  X_test.loc[X_test.index[0],["shift_PCT_1"]][0],
                                  X_test.loc[X_test.index[0],["shift_PCT_2"]][0],
                                   X_test.loc[X_test.index[0],["shift_PCT_3"]][0],rms,rms05]

        idx += 1

test_pred_df[['y','y_pred',"shift_PCT_1","shift_PCT_2","shift_PCT_3",'rms','rms0.5']] = test_pred_df[['y','y_pred',"shift_PCT_1","shift_PCT_2","shift_PCT_3",'rms','rms0.5']].astype(float)

2016LG =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 1s 55ms/sample - loss: 1.7068 - mae: 0.9164
Epoch 2/100
25/25 [==============================] - 0s 2ms/sample - loss: 0.0124 - mae: 0.0886
Epoch 3/100
25/25 [==============================] - 0s 2ms/sample - loss: 0.0099 - mae: 0.0745
Epoch 4/100
25/25 [==============================] - 0s 2ms/sample - loss: 0.0042 - mae: 0.0526
Epoch 5/100
25/25 [==============================] - 0s 2ms/sa

25/25 [==============================] - 1s 58ms/sample - loss: 2.1584 - mae: 1.0008
Epoch 2/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0472 - mae: 0.2002
Epoch 3/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0121 - mae: 0.0948
Epoch 4/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0023 - mae: 0.0379
Epoch 5/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0027 - mae: 0.0409
Epoch 6/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0013 - mae: 0.0292
Epoch 7/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0016 - mae: 0.0327
Epoch 8/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0016 - mae: 0.0334
Epoch 00008: early stopping
2016WO =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)     

25/25 [==============================] - 0s 3ms/sample - loss: 0.0112 - mae: 0.0908
Epoch 5/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0108 - mae: 0.0895
Epoch 6/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0105 - mae: 0.0884
Epoch 7/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0116 - mae: 0.0934
Epoch 8/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0120 - mae: 0.0935
Epoch 00008: early stopping
2017HH =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_____________________________________

25/25 [==============================] - 0s 3ms/sample - loss: 0.0024 - mae: 0.0376 0s - loss: 0.0026 - mae: 0.038
Epoch 12/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0024 - mae: 0.0423
Epoch 13/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0024 - mae: 0.0369
Epoch 00013: early stopping
2017WO =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 1s 55ms/sample - loss: 1.3673 - mae: 0.8949
Epoch 2/100
2

25/25 [==============================] - 0s 3ms/sample - loss: 0.0033 - mae: 0.0459
Epoch 10/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0036 - mae: 0.0482
Epoch 00010: early stopping
2018HH =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 1s 58ms/sample - loss: 1.4394 - mae: 0.7555
Epoch 2/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0245 - mae: 0.1338
Epoch 3/100
25/25 [==========================

25/25 [==============================] - 0s 2ms/sample - loss: 0.0047 - mae: 0.0602
Epoch 8/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0050 - mae: 0.0618
Epoch 00008: early stopping
2018KT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               63200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 25 samples
Epoch 1/100
25/25 [==============================] - 1s 58ms/sample - loss: 1.0918 - mae: 0.8099
Epoch 2/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0313 - mae: 0.1534
Epoch 3/100
25/25 [===========================

25/25 [==============================] - 2s 69ms/sample - loss: 1.6433 - mae: 0.9430
Epoch 2/100
25/25 [==============================] - 0s 5ms/sample - loss: 0.0103 - mae: 0.0905
Epoch 3/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0057 - mae: 0.0595
Epoch 4/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0021 - mae: 0.0386
Epoch 5/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.0013 - mae: 0.0299
Epoch 6/100
25/25 [==============================] - 0s 5ms/sample - loss: 0.0013 - mae: 0.0300
Epoch 7/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0012 - mae: 0.0278
Epoch 8/100
25/25 [==============================] - 0s 2ms/sample - loss: 0.0011 - mae: 0.0265
Epoch 9/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0012 - mae: 0.0281
Epoch 10/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0011 - mae: 0.0270
Epoch 00010: early stopping
2019HH ===============

25/25 [==============================] - 2s 65ms/sample - loss: 1.2106 - mae: 0.8441
Epoch 2/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0198 - mae: 0.1244
Epoch 3/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0107 - mae: 0.0869
Epoch 4/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0065 - mae: 0.0695
Epoch 5/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0055 - mae: 0.0652
Epoch 6/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0050 - mae: 0.0629 0s - loss: 0.0046 - mae: 0.060
Epoch 7/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0056 - mae: 0.0655
Epoch 8/100
25/25 [==============================] - 0s 3ms/sample - loss: 0.0054 - mae: 0.0629
Epoch 00008: early stopping
2019WO =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape          

In [9]:
# # 100, lr = 0.01, batch 2
# test_pred_df.sort_values(by=["YEAR","T_ID"]).to_csv("PCT_3.csv",index = False
#                                                 )

In [11]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
YEAR,,,,,,
2016,0.492391,0.495652,0.504167,0.508514,0.070957,0.075725
2017,0.498370,0.489564,0.491848,0.521937,0.122198,0.076630
2018,0.507971,0.496652,0.500362,0.497480,0.095594,0.071377
2019,0.505072,0.498990,0.508152,0.495290,0.096964,0.105072


In [14]:
mean_squared_error(test_pred_df['y'],test_pred_df['y_pred'])

0.015411139084997588

In [12]:
test_pred_df

,YEAR,T_ID,y,y_pred,shift_PCT_1,shift_PCT_2,shift_PCT_3,rms,rms0.5
0,2016,LG,0.608696,0.629710,0.583333,0.458333,0.375000,0.021015,0.108696
1,2016,HH,0.500000,0.522560,0.500000,0.608696,0.521739,0.022560,0.000000
2,2016,NC,0.565217,0.519151,0.500000,0.541667,0.739130,0.046066,0.065217
3,2016,HT,0.458333,0.572146,0.500000,0.583333,0.458333,0.113812,0.041667
4,2016,SK,0.458333,0.432622,0.458333,0.458333,0.458333,0.025711,0.041667
5,2016,KT,0.291667,0.306353,0.333333,0.375000,0.416667,0.014687,0.208333
6,2016,WO,0.375000,0.563867,0.625000,0.666667,0.500000,0.188867,0.125000
7,2016,LT,0.500000,0.403313,0.375000,0.500000,0.458333,0.096687,0.000000
8,2016,SS,0.500000,0.499762,0.500000,0.434783,0.333333,0.000238,0.000000
9,2016,OB,0.666667,0.520579,0.666667,0.458333,0.666667,0.146087,0.166667


## +)

In [7]:
# 100, lr = 0.01, batch 10
test_pred_df

,YEAR,T_ID,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
0,2016,LG,0.608696,0.324510,0.583333,0.458333,0.284185,0.108696
1,2016,HH,0.500000,0.620822,0.500000,0.608696,0.120822,0.000000
2,2016,NC,0.565217,0.315737,0.500000,0.541667,0.249481,0.065217
3,2016,HT,0.458333,0.732308,0.500000,0.583333,0.273975,0.041667
4,2016,SK,0.458333,0.608401,0.458333,0.458333,0.150068,0.041667
5,2016,KT,0.291667,0.362899,0.333333,0.375000,0.071232,0.208333
6,2016,WO,0.375000,0.764267,0.625000,0.666667,0.389267,0.125000
7,2016,LT,0.500000,0.532504,0.375000,0.500000,0.032504,0.000000
8,2016,SS,0.500000,0.655361,0.500000,0.434783,0.155361,0.000000
9,2016,OB,0.666667,0.320553,0.666667,0.458333,0.346113,0.166667


In [8]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
YEAR,,,,,,
2016,0.492391,0.523736,0.504167,0.508514,0.207301,0.075725
2017,0.498370,0.346776,0.491848,0.521937,0.245242,0.076630
2018,0.507971,0.574614,0.500362,0.497480,0.212128,0.071377
2019,0.505072,0.506905,0.508152,0.495290,0.275356,0.105072


In [15]:
# 100, lr = 0.01, batch 5
test_pred_df

,YEAR,T_ID,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
0,2016,LG,0.608696,0.703705,0.583333,0.458333,0.095009,0.108696
1,2016,HH,0.500000,0.505495,0.500000,0.608696,0.005495,0.000000
2,2016,NC,0.565217,0.590402,0.500000,0.541667,0.025184,0.065217
3,2016,HT,0.458333,0.552457,0.500000,0.583333,0.094123,0.041667
4,2016,SK,0.458333,0.508638,0.458333,0.458333,0.050305,0.041667
5,2016,KT,0.291667,0.410087,0.333333,0.375000,0.118421,0.208333
6,2016,WO,0.375000,0.706302,0.625000,0.666667,0.331302,0.125000
7,2016,LT,0.500000,0.345885,0.375000,0.500000,0.154115,0.000000
8,2016,SS,0.500000,0.630994,0.500000,0.434783,0.130994,0.000000
9,2016,OB,0.666667,0.577408,0.666667,0.458333,0.089259,0.166667


In [16]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
YEAR,,,,,,
2016,0.492391,0.553137,0.504167,0.508514,0.109421,0.075725
2017,0.498370,0.519487,0.491848,0.521937,0.125158,0.076630
2018,0.507971,0.524504,0.500362,0.497480,0.085791,0.071377
2019,0.505072,0.518788,0.508152,0.495290,0.109644,0.105072


In [11]:
# 100, lr = 0.01, batch 2
test_pred_df

,YEAR,T_ID,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
0,2016,LG,0.608696,0.639421,0.583333,0.458333,0.030725,0.108696
1,2016,HH,0.500000,0.519752,0.500000,0.608696,0.019752,0.000000
2,2016,NC,0.565217,0.521629,0.500000,0.541667,0.043589,0.065217
3,2016,HT,0.458333,0.557003,0.500000,0.583333,0.098669,0.041667
4,2016,SK,0.458333,0.434201,0.458333,0.458333,0.024132,0.041667
5,2016,KT,0.291667,0.321546,0.333333,0.375000,0.029879,0.208333
6,2016,WO,0.375000,0.588199,0.625000,0.666667,0.213199,0.125000
7,2016,LT,0.500000,0.385834,0.375000,0.500000,0.114166,0.000000
8,2016,SS,0.500000,0.500726,0.500000,0.434783,0.000726,0.000000
9,2016,OB,0.666667,0.495426,0.666667,0.458333,0.171241,0.166667


In [12]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
YEAR,,,,,,
2016,0.492391,0.496374,0.504167,0.508514,0.074608,0.075725
2017,0.498370,0.488359,0.491848,0.521937,0.112179,0.076630
2018,0.507971,0.487128,0.500362,0.497480,0.092531,0.071377
2019,0.505072,0.506926,0.508152,0.495290,0.098745,0.105072


In [9]:
# 100, lr = 0.01, batch 1
test_pred_df

,YEAR,T_ID,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
0,2016,LG,0.608696,0.640221,0.583333,0.458333,0.031525,0.108696
1,2016,HH,0.500000,0.551892,0.500000,0.608696,0.051892,0.000000
2,2016,NC,0.565217,0.487683,0.500000,0.541667,0.077534,0.065217
3,2016,HT,0.458333,0.545832,0.500000,0.583333,0.087499,0.041667
4,2016,SK,0.458333,0.436860,0.458333,0.458333,0.021474,0.041667
5,2016,KT,0.291667,0.348948,0.333333,0.375000,0.057281,0.208333
6,2016,WO,0.375000,0.560369,0.625000,0.666667,0.185369,0.125000
7,2016,LT,0.500000,0.352799,0.375000,0.500000,0.147201,0.000000
8,2016,SS,0.500000,0.483279,0.500000,0.434783,0.016721,0.000000
9,2016,OB,0.666667,0.516517,0.666667,0.458333,0.150149,0.166667


In [8]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_PCT_1,shift_PCT_2,rms,rms0.5
YEAR,,,,,,
2016,0.492391,0.492440,0.504167,0.508514,0.082665,0.075725
2017,0.498370,0.483163,0.491848,0.521937,0.119453,0.076630
2018,0.507971,0.504695,0.500362,0.497480,0.088655,0.071377
2019,0.505072,0.497719,0.508152,0.495290,0.117082,0.105072


#### +======================================+

In [ ]:
test_pred_df #300, lr = 0.01

In [ ]:
test_pred_df #300, lr = 0.001

In [ ]:
test_pred_df #300, lr = 0.0005

In [ ]:
test_pred_df #100, lr = 0.1

In [ ]:
test_pred_df #100, lr = 0.01, patience 3

In [ ]:
test_pred_df #100, lr = 0.01, patience 1

In [ ]:
 #100, lr = 0.01, patience 2 +++, rms ver, rms05 ver, Adam

test_pred_df

In [ ]:
 #100, lr = 0.01, patience 2 +++, rms ver, rms05 ver
# model_copy = model_dict.copy()
# hist_copy = hist_dict.copy()
# tmp_df = test_pred_df.copy()
tmp_df

In [ ]:
test_pred_df #100, lr = 0.01, patience 2 +++, rms ver

In [ ]:
test_pred_df #100, lr = 0.01, patience 2 ++

In [ ]:
test_pred_df #100, lr = 0.01, patience 2

In [ ]:
test_pred_df #100, lr = 0.01

In [ ]:
test_pred_df #100, lr = 0.001

In [ ]:
test_pred_df #100, lr = 0.001

In [ ]:
test_pred_df #100, lr=0.0005

In [ ]:
test_pred_df #200, lr = 0.01

In [ ]:
test_pred_df #200, lr = 0.001

In [ ]:
test_pred_df #200, lr=0.0005

In [ ]:
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist1.history['loss'], 'y', label='train loss')

acc_ax.plot(hist1.history['mae'], 'b', label='train mae')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('mae')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()